<a href="https://colab.research.google.com/github/JimKing100/Jestimate/blob/master/SF_Map_Code5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The San Francisco Real Estate Project

In [1]:
# Install fiona - need to comment out for transfer to live site.
# Turn on for running in a notebook
#%%capture
#!pip install fiona

     |████████████████████████████████| 11.9MB 2.7MB/s 
  Created wheel for munch: filename=munch-2.3.2-py2.py3-none-any.whl size=6614 sha256=4d7fed6084a649bacf90588caa3bd4ca66cf7477ee87a05837919d45fa3957c4
  Stored in directory: /root/.cache/pip/wheels/db/bf/bc/06a3e1bfe0ab27d2e720ceb3cff3159398d92644c0cec2c125
Successfully built munch


In [2]:
# Install geopandas - need to comment out for tranfer to live site.
# Turn on for running in a notebook
#%%capture
#!pip install geopandas

     |████████████████████████████████| 921kB 2.8MB/s 
     |████████████████████████████████| 10.1MB 37.9MB/s 


In [0]:
# Import libraries
import pandas as pd
import numpy as np
import math

import geopandas
import json

from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, NumeralTickFormatter
from bokeh.palettes import brewer

from bokeh.io.doc import curdoc
from bokeh.models import Slider, HoverTool, Select, TapTool, CustomJS, ColumnDataSource, TableColumn, DataTable, CDSView, GroupFilter
from bokeh.layouts import widgetbox, row, column, gridplot
from bokeh.models.widgets import TextInput

### Load and Clean the Data

In [4]:
# Load SF real estate data - 10 years (2009-2018) of single family home sales in San Francisco downloaded from the SF MLS
# Longitude and latitude were added to the csv file prior to loading using geocoding.geo.census.gov 
initial_data = pd.read_csv('https://raw.githubusercontent.com/JimKing100/SF_Real_Estate_Live/master/data/SF-SFR-Sales-Final1.csv')
results_data = pd.read_csv('https://raw.githubusercontent.com/JimKing100/Jestimate/master/data/display_data.csv')

results_data = results_data.drop(results_data.index[1153])

results_data['full_address'] = results_data.apply(lambda x: x['full_address'].split(',')[0], axis=1)
results_data['neighborhood'] = results_data.apply(lambda x: x['neighborhood'].split('- ')[1], axis=1)

# Rename subdistr_desc to neighborhood
temp_data = initial_data.rename(columns={'subdist_desc': 'neighborhood'})

# Check the data
print(temp_data.shape)
temp_data.head(5)

(23711, 30)


,longitude,latitude,full_address,city,state,street_no,street_name,street_nox,street_suffix,zip,area,district_no,district_desc,subdist_no,neighborhood,on_market_data,cdom,orig_list_price,sale_date,sale_price,rooms,baths,beds,sf_source,sf_source_decs,sf,lot_acres,lot_sf,year_built,zoning
0,-122.466620,37.753273,"1878 10th Ave, San Francisco, CA 94122-4602",San Francisco,CA,1878,10th,NaN,Ave,94122,2060,2,SF District 2,2060,2 - Inner Sunset,8/31/11,65,649000,12/22/11,580000,6,1.0,2,T,Per Tax Records,1480,NaN,0,1932,RH1
1,-122.466670,37.753850,"1842 10th Ave, San Francisco, CA 94122-4602",San Francisco,CA,1842,10th,NaN,Ave,94122,2060,2,SF District 2,2060,2 - Inner Sunset,10/8/08,164,679000,3/23/09,509000,0,1.0,2,T,Per Tax Records,920,NaN,0,1908,RH1
2,-122.466700,37.754070,"1829 10th Ave, San Francisco, CA 94122",San Francisco,CA,1829,10th,NaN,Ave,94122,2060,2,SF District 2,2060,2 - Inner Sunset,9/6/13,15,699000,10/11/13,840000,0,1.5,2,T,Per Tax Records,1200,0.0693,3017,1906,RH1
3,-122.466965,37.758057,"1612 10th Ave, San Francisco, CA 94122",San Francisco,CA,1612,10th,NaN,Ave,94122,2060,2,SF District 2,2060,2 - Inner Sunset,6/8/10,9,699900,7/29/10,817000,6,1.0,3,T,Per Tax Records,1800,NaN,0,1914,NaN
4,-122.466910,37.757130,"1671 10th Ave, San Francisco, CA 94122",San Francisco,CA,1671,10th,NaN,Ave,94122,2060,2,SF District 2,2060,2 - Inner Sunset,9/8/11,25,749000,10/6/11,785000,5,1.0,2,T,Per Tax Records,1050,NaN,0,1938,RH2


In [0]:
# Fill house square foot zero values with the average house square footage by bedroom for all single family homes in SF
averagesf_data = temp_data.groupby('beds').sf.mean()

# Use average sf by bedroom for each 0 value in each bedroom group up to 9 bedrooms
for i in range(0, 9): 
  temp_data.loc[(temp_data['sf'] == 0) & (temp_data['beds'] == i), 'sf'] = averagesf_data.loc[i]

# Use 10,000sf for anything over 9 bedrooms
temp_data.loc[temp_data['sf'] == 0, 'sf'] = 10000
temp_data = temp_data.astype({'sf': int})

In [0]:
# Map MLS neighborhood code (e.g. 1010) to GeoDataFrame neighborhood id (e.g. '1a') to create the sf_data for all 10 years
neighborhood_dict = {1010: '1a', 1020: '1b', 1030: '1c', 1040: '1d', 1050: '1e', 1060: '1f', 1070: '1g',
                     2010: '2a', 2020: '2b', 2030: '2c', 2040: '2d', 2050: '2e', 2060: '2f', 2070: '2g',
                     3010: '3a', 3020: '3b', 3030: '3c', 3040: '3d', 3050: '3e', 3060: '3f', 3070: '3g', 3080: '3h', 3090: '3j',
                     4010: '4a', 4020: '4b', 4030: '4c', 4040: '4d', 4050: '4e', 4060: '4f', 4070: '4g', 4080: '4h', 4090: '4j', 4100: '4k', 4110: '4m', 4120: '4n', 4130: '4p', 4140: '4r', 4150: '4s', 4160: '4t',
                     5010: '5a', 5020: '5b', 5030: '5c', 5040: '5d', 5050: '5e', 5060: '5f', 5070: '5g', 5080: '5h', 5090: '5j', 5100: '5k', 5110: '5m',
                     6010: '6a', 6020: '6b', 6030: '6c', 6040: '6d', 6050: '6e', 6060: '6f',
                     7010: '7a', 7020: '7b', 7030: '7c', 7040: '7d', 
                     8010: '8a', 8020: '8b', 8030: '8c', 8040: '8d', 8050: '8e', 8060: '8f', 8070: '8g', 8080: '8h', 8090: '8j',
                     9010: '9a', 9020: '9c', 9030: '9d', 9040: '9e', 9050: '9f', 9060: '9h', 9070: '9j', 9080: '9g', 
                     10010: '10a', 10020: '10b', 10030: '10c', 10040: '10d', 10050: '10e', 10060: '10f', 10070: '10g', 10080: '10h', 10090: '10j', 10100: '10k', 10110: '10m', 10120: '10n'
                    }
sf_data = temp_data.replace({'subdist_no': neighborhood_dict})

In [7]:
# Create a year column based on the sale date
sf_data['year'] = '20' + sf_data['sale_date'].str[-2:]
sf_data.head(5)

# Create a price_sf column
sf_data['price_sf'] = sf_data['sale_price'] / sf_data['sf']
sf_data.head(5)

,longitude,latitude,full_address,city,state,street_no,street_name,street_nox,street_suffix,zip,area,district_no,district_desc,subdist_no,neighborhood,on_market_data,cdom,orig_list_price,sale_date,sale_price,rooms,baths,beds,sf_source,sf_source_decs,sf,lot_acres,lot_sf,year_built,zoning,year,price_sf
0,-122.466620,37.753273,"1878 10th Ave, San Francisco, CA 94122-4602",San Francisco,CA,1878,10th,NaN,Ave,94122,2060,2,SF District 2,2f,2 - Inner Sunset,8/31/11,65,649000,12/22/11,580000,6,1.0,2,T,Per Tax Records,1480,NaN,0,1932,RH1,2011,391.891892
1,-122.466670,37.753850,"1842 10th Ave, San Francisco, CA 94122-4602",San Francisco,CA,1842,10th,NaN,Ave,94122,2060,2,SF District 2,2f,2 - Inner Sunset,10/8/08,164,679000,3/23/09,509000,0,1.0,2,T,Per Tax Records,920,NaN,0,1908,RH1,2009,553.260870
2,-122.466700,37.754070,"1829 10th Ave, San Francisco, CA 94122",San Francisco,CA,1829,10th,NaN,Ave,94122,2060,2,SF District 2,2f,2 - Inner Sunset,9/6/13,15,699000,10/11/13,840000,0,1.5,2,T,Per Tax Records,1200,0.0693,3017,1906,RH1,2013,700.000000
3,-122.466965,37.758057,"1612 10th Ave, San Francisco, CA 94122",San Francisco,CA,1612,10th,NaN,Ave,94122,2060,2,SF District 2,2f,2 - Inner Sunset,6/8/10,9,699900,7/29/10,817000,6,1.0,3,T,Per Tax Records,1800,NaN,0,1914,NaN,2010,453.888889
4,-122.466910,37.757130,"1671 10th Ave, San Francisco, CA 94122",San Francisco,CA,1671,10th,NaN,Ave,94122,2060,2,SF District 2,2f,2 - Inner Sunset,9/8/11,25,749000,10/6/11,785000,5,1.0,2,T,Per Tax Records,1050,NaN,0,1938,RH2,2011,747.619048


### Create the Summary Data by Neighborhood

In [8]:
# Function to calculate the monthly housing payment (PITI)
def minimum_income(median_price):
    int_rate = .04
    term = 30
    down_pmt = .20
    principal_pmt = median_price * (1 - down_pmt)
    
    # Calculate insurance and taxes
    tax_pmt = (median_price * .01) / 12
    insurance_pmt = (median_price * .0038) / 12
    
    # monthly rate from annual percentage rate
    interest_rate = int_rate/(100 * 12)
    
    # total number of payments
    payment_num = term * 12
    
    # calculate monthly mortgage payment
    mortgage_pmt = principal_pmt * (interest_rate * (math.pow((1 + interest_rate), (payment_num))) / 
                               (math.pow((1 + interest_rate), (payment_num)) - 1))
    payment = mortgage_pmt + tax_pmt + insurance_pmt
    min_income = (payment / .30) * 12
    
    return min_income

neighborhood_data = sf_data.groupby(
    ['year', 'subdist_no']
).agg(
    {
    'sale_price': ['count', 'mean', 'median'],
    'sf': ['mean'],
    'price_sf': ['mean']
    }
)

#Reset the index to 1 level to fill in year
neighborhood_data = neighborhood_data.set_axis(neighborhood_data.columns.map('_'.join), axis=1, inplace=False)
neighborhood_data = neighborhood_data.reset_index(level=[0,1])

# Change data types to integer for price_sf and year
neighborhood_data = neighborhood_data.astype({'sale_price_mean': 'int'})
neighborhood_data = neighborhood_data.astype({'sale_price_median': 'int'})
neighborhood_data = neighborhood_data.astype({'sf_mean': 'int'})
neighborhood_data = neighborhood_data.astype({'price_sf_mean': 'int'})
neighborhood_data = neighborhood_data.astype({'year': 'int'})

# Add the monthly income required to buy the median house in the neighborhood
neighborhood_data['min_income'] = neighborhood_data.apply(lambda x: minimum_income(x['sale_price_median']), axis=1)
neighborhood_data = neighborhood_data.astype({'min_income': 'int'})

neighborhood_data.head()

,year,subdist_no,sale_price_count,sale_price_mean,sale_price_median,sf_mean,price_sf_mean,min_income
0,2009,10a,92,351929,350000,1366,287,47398
1,2009,10b,69,541515,545000,1360,420,73806
2,2009,10c,98,509623,520000,1208,440,70420
3,2009,10d,46,564891,590000,1380,438,79900
4,2009,10e,80,460139,456000,1390,355,61753


### Prepare the mapping data and GeoDataFrame

In [9]:
# Read the geojson map file for Realtor Neighborhoods into a GeoDataframe object
sf = geopandas.read_file('https://raw.githubusercontent.com/JimKing100/SF_Real_Estate_Live/master/data/Realtor%20Neighborhoods.geojson')

# Set the Coordinate Referance System (crs) for projections
# ESPG code 4326 is also referred to as WGS84 lat-long projection
sf.crs = {'init': 'epsg:4326'}

# Rename columns in geojson map file
sf = sf.rename(columns={'geometry': 'geometry','nbrhood':'neighborhood_name', 'nid': 'subdist_no'}).set_geometry('geometry')

# Change neighborhood id (subdist_no) for correct code for Mount Davidson Manor and for parks
sf.loc[sf['neighborhood_name'] == 'Mount Davidson Manor', 'subdist_no'] = '4n'
sf.loc[sf['neighborhood_name'] == 'Golden Gate Park', 'subdist_no'] = '12a'
sf.loc[sf['neighborhood_name'] == 'Presidio', 'subdist_no'] = '12b'
sf.loc[sf['neighborhood_name'] == 'Lincoln Park', 'subdist_no'] = '12c'

sf.sort_values(by=['subdist_no'])
sf.head()

,neighborhood_name,subdist_no,sfar_distr,geometry
0,Alamo Square,6e,District 6 - Central North,"MULTIPOLYGON (((-122.42948 37.77510, -122.4310..."
1,Anza Vista,6a,District 6 - Central North,"MULTIPOLYGON (((-122.44746 37.77986, -122.4473..."
2,Balboa Terrace,4a,District 4 - Twin Peaks West,"MULTIPOLYGON (((-122.46451 37.73221, -122.4650..."
3,Bayview,10a,District 10 - Southeast,"MULTIPOLYGON (((-122.38759 37.75026, -122.3874..."
4,Bernal Heights,9a,District 9 - Central East,"MULTIPOLYGON (((-122.40375 37.74919, -122.4046..."


### Create colorbar formatting lookup table

In [10]:
# This dictionary contains the formatting for the data in the plots
format_data = [('sale_price_count', 0, 100,'0,0', 'Number of Sales'),
               ('sale_price_mean', 500000, 4000000,'$0,0', 'Average Sales Price'),
               ('sale_price_median', 500000, 4000000, '$0,0', 'Median Sales Price'),
               ('sf_mean', 500, 5000,'0,0', 'Average Square Footage'),
               ('price_sf_mean', 0, 2000,'$0,0', 'Average Price Per Square Foot'),
               ('min_income', 50000, 600000,'$0,0', 'Minimum Income Required')
              ]
 
#Create a DataFrame object from the dictionary 
format_df = pd.DataFrame(format_data, columns = ['field' , 'min_range', 'max_range' , 'format', 'verbage'])
format_df.head(10)

,field,min_range,max_range,format,verbage
0,sale_price_count,0,100,"0,0",Number of Sales
1,sale_price_mean,500000,4000000,"$0,0",Average Sales Price
2,sale_price_median,500000,4000000,"$0,0",Median Sales Price
3,sf_mean,500,5000,"0,0",Average Square Footage
4,price_sf_mean,0,2000,"$0,0",Average Price Per Square Foot
5,min_income,50000,600000,"$0,0",Minimum Income Required


### Create the Interactive Plot

In [0]:
# Create a function the returns json_data for the year selected by the user
def json_data(selectedYear):
    yr = selectedYear
    
    # Pull selected year from neighborhood summary data
    df_yr = neighborhood_data[neighborhood_data['year'] == yr]
    
    # Merge the GeoDataframe object (sf) with the neighborhood summary data (neighborhood)
    merged = pd.merge(sf, df_yr, on='subdist_no', how='left')
    
    # Fill the null values
    values = {'year': yr, 'sale_price_count': 0, 'sale_price_mean': 0, 'sale_price_median': 0,
              'sf_mean': 0, 'price_sf_mean': 0, 'min_income': 0}
    merged = merged.fillna(value=values)
    
    # Bokeh uses geojson formatting, representing geographical features, with json
    # Convert to json
    merged_json = json.loads(merged.to_json())
    
    # Convert to json preferred string-like object 
    json_data = json.dumps(merged_json)
    return json_data
  
# Define the callback function: update_plot
def update_plot(yr, cr):
    # The input yr is the year selected from the slider
    #yr = slider.value
    new_data = json_data(yr)
    
    # The input cr is the criteria selected from the select box
    #cr = select.value
    input_field = format_df.loc[format_df['verbage'] == cr, 'field'].iloc[0]
    
    # Update the plot based on the changed inputs
    p = make_plot(input_field)
    
    # Update the layout, clear the old document and display the new document
    layout = column(p, tap_neighborhood)
    curdoc().clear()
    curdoc().add_root(layout)
    
    # Update the data
    geosource.geojson = new_data
    
# Create a plotting function
def make_plot(field_name):    
  # Set the format of the colorbar
  min_range = format_df.loc[format_df['field'] == field_name, 'min_range'].iloc[0]
  max_range = format_df.loc[format_df['field'] == field_name, 'max_range'].iloc[0]
  field_format = format_df.loc[format_df['field'] == field_name, 'format'].iloc[0]

  # Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
  color_mapper = LinearColorMapper(palette = palette, low = min_range, high = max_range)

  # Create color bar.
  format_tick = NumeralTickFormatter(format=field_format)
  color_bar = ColorBar(color_mapper=color_mapper, label_standoff=18, formatter=format_tick,
  border_line_color=None, location = (0, 0))

  # Create figure object.
  verbage = format_df.loc[format_df['field'] == field_name, 'verbage'].iloc[0]

  p = figure(title = verbage + ' by Neighborhood for Single Family Homes in SF by Year - 2009 to 2018', 
             plot_height = 650, plot_width = 850,
             toolbar_location = None)
  p.xgrid.grid_line_color = None
  p.ygrid.grid_line_color = None
  p.axis.visible = False

  # Add patch renderer to figure. 
  p.patches('xs','ys', source = geosource, fill_color = {'field' : field_name, 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
  
  # Specify color bar layout.
  p.add_layout(color_bar, 'right')

  # Add the hover tool to the graph
  p.add_tools(hover)
  
  return p
    
    
# Input geojson source that contains features for plotting for:
# initial year 2018 and initial criteria sale_price_median
geosource = GeoJSONDataSource(geojson = json_data(2018))
input_field = 'sale_price_mean'

# Input the results data
source = ColumnDataSource(results_data)
hood = 'Marina'#tap_neighborhood.value
view1 = CDSView(source=source, filters=[GroupFilter(column_name='neighborhood', group=hood)])
columns = [TableColumn(field = 'full_address', title = 'Address')]

# Define a sequential multi-hue color palette.
palette = brewer['Blues'][8]

# Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]

callback = CustomJS(args=dict(source=geosource), code="""
    console.log(cb_obj)
    console.log(cb_data)
    x = cb_data.geometries['x']
    y = cb_data.geometries['y']
    z = cb_data.source.attributes.data.neighborhood_name[66]
    console.log(x)
    console.log(y)
    console.log(z)
""")

#Add hover tool
hover = HoverTool(tooltips = [ ('Neighborhood','@neighborhood_name'),
                               ('# Sales', '@sale_price_count'),
                               ('Average Price', '$@sale_price_mean{,}'),
                               ('Median Price', '$@sale_price_median{,}'),
                               ('Average SF', '@sf_mean{,}'),
                               ('Price/SF ', '$@price_sf_mean{,}'),
                               ('Income Needed', '$@min_income{,}')])

# Add tap tool
#tap = TapTool(callback=callback)

# Call the plotting function
p = make_plot(input_field)

columns = [TableColumn(field = 'full_address', title = 'Address')]

data_table = DataTable(source = source, view = view1, columns = columns, width = 280, height = 280, editable = False)
tap_neighborhood = TextInput(value = 'Marina', title = 'Neighborhood')
table_address = TextInput(value = '', title = 'Address')
table_actual = TextInput(value = '', title = 'Actual Sale Price')
table_predicted = TextInput(value = '', title = 'Predicted Sale Price')
table_diff = TextInput(value = '', title = 'Difference')

def function_source(attr, old, new):
    try:
        selected_index = source.selected.indices[0]
        table_address.value = str(source.data['full_address'][selected_index])
        table_actual.value = '${:,}'.format((source.data['sale_price'][selected_index]))
        table_predicted.value = '${:,}'.format((source.data['prediction'][selected_index]))
        table_diff.value = '${:,}'.format(source.data['difference'][selected_index])
    except IndexError:
        pass

source.selected.on_change('indices', function_source)

# # Add text fields
# tap_neighborhood = TextInput(value = 'Marina', title = 'Neighborhood')
# data_table = DataTable(source = source, view = view1, columns = columns, width = 280, height = 280, editable = False)
# table_address = TextInput(value = '', title = 'Address')
# table_actual = TextInput(value = '', title = 'Actual Sale Price')
# table_predicted = TextInput(value = '', title = 'Predicted Sale Price')
# table_diff = TextInput(value = '', title = 'Difference')

# def function_source(attr, old, new):
#     try:
#         selected_index = geosource.selected.indices[0]
#         tap_neighborhood.value = sf.iloc[selected_index]['neighborhood_name']#str(selected_index)#str(selected_index)#results_data.iloc[selected_index]['neighborhood'] #str(selected_index)#str(selected_index)
#         hood = tap_neighborhood.value
#         view1 = CDSView(source=source, filters=[GroupFilter(column_name='neighborhood', group=hood)])
#         columns = [TableColumn(field = 'full_address', title = 'Address')
#         selected_index = source.selected.indices[0]
#         table_address.value = str(source.data['full_address'][selected_index])
#         table_actual.value = '${:,}'.format((source.data['sale_price'][selected_index]))
#         table_predicted.value = '${:,}'.format((source.data['prediction'][selected_index]))
# #         table_diff.value = '${:,}'.format(source.data['difference'][selected_index])
#     except IndexError:
#         pass 

#geosource.selected.on_change('indices', function_source)
#update_plot(2018, 'Average Sales Price')

# Make a slider object: slider 
#slider = Slider(title = 'Year',start = 2009, end = 2018, step = 1, value = 2018)
#slider.on_change('value', update_plot)

# Make a selection object: select
#select = Select(title='Select Criteria:', value='Median Sales Price', options=['Median Sales Price', 'Minimum Income Required',
#                                                                               'Average Sales Price', 'Average Price Per Square Foot',
#                                                                               'Average Square Footage', 'Number of Sales'])
#select.on_change('value', update_plot)

#source.selected.on_change('indices', function_source)
#curdoc().add_root(column(data_table, table_address, table_actual, table_predicted, table_diff))

# Make a column layout of widgetbox(slider) and plot, and add it to the current document
# Display the current document
#layout = column(p, tap_neighborhood)

curdoc().add_root(gridplot(
                           [[p, column(tap_neighborhood, data_table, table_address, table_actual, table_predicted, table_diff)]],
                           toolbar_location = None))

# Use the following code to test in a notebook
# Interactive features will no show in notebook
#output_notebook()
#show(p)